In [ ]:
import nltk
import csv
import numpy as np
from nltk.corpus import wordnet as wn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

final_path='/home/ambika/INF_project/final_image_paths.txt'
image_list = np.loadtxt(final_path, str)

filename='detection_results.txt'
b = open(filename, 'wb') 
a = csv.writer(b)

for p in range(0,1000):
    !mkdir -p _temp
    !echo ''{image_list[p]}'' > _temp/det_input.txt

    !/home/ambika/caffe/python/detect.py --crop_mode=selective_search --pretrained_model=/home/ambika/caffe/models/bvlc_reference_rcnn_ilsvrc13/bvlc_reference_rcnn_ilsvrc13.caffemodel --model_def=/home/ambika/caffe/models/bvlc_reference_rcnn_ilsvrc13/deploy.prototxt --raw_scale=255 /home/ambika/INF_project/_temp/det_input.txt /home/ambika/INF_project/_temp/det_output.h5    

    df = pd.read_hdf('/home/ambika/INF_project/_temp/det_output.h5', 'df')
#print(df.shape)
#print(df.iloc[0])
#print(df.iloc[1])

    with open('/home/ambika/caffe/data/ilsvrc12/det_synset_words.txt') as f:
        labels_df = pd.DataFrame([
            {
                'synset_id': l.strip().split(' ')[0],
                'name': ' '.join(l.strip().split(' ')[1:]).split(',')[0]
            }
            for l in f.readlines()
        ])
    labels_df.sort('synset_id')
    predictions_df = pd.DataFrame(np.vstack(df.prediction.values), columns=labels_df['name'])
#print(predictions_df.iloc[0])

    max_s = predictions_df.max(0)
    max_s.sort(ascending=False)
#print(max_s[:10])
#print(max_s[:1])
    x=max_s.axes
    y=x[0][0]
#print y
    y2=x[0][1]
#print y2

# Find, print, and display the top detections: person and bicycle.
    i = predictions_df[y].argmax()
    j = predictions_df[y2].argmax()

# Show top predictions for top detection.
    f = pd.Series(df['prediction'].iloc[i], index=labels_df['name'])
#print('Top detection:')
#print(f.order(ascending=False)[:2])
    f1=f.order(ascending=False)[:2]
    f_ind1=f1.axes
#print f_ind1[0][0]
#print f1[0]
#print('')

# Show top predictions for second-best detection.
    f = pd.Series(df['prediction'].iloc[j], index=labels_df['name'])
#print('Second-best detection:')
#print(f.order(ascending=False)[:2])
    f2=f.order(ascending=False)[:2]
    f_ind2=f2.axes
#print f_ind2[0][0]
#print f2[0]

# Show top detection in red, second-best top detection in blue.
    im = plt.imread(image_list[p])
    plt.imshow(im)
    currentAxis = plt.gca()

    det1 = df.iloc[i]
    print det1
#coords = (det1['xmin'], det1['ymin']), det1['xmax'] - det1['xmin'], det1['ymax'] - det1['ymin']
#currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor='r', linewidth=5))

    det2 = df.iloc[j]
    print det2
#coords = (det2['xmin'], det2['ymin']), det2['xmax'] - det2['xmin'], det2['ymax'] - det2['ymin']
#currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor='b', linewidth=5))

#if (det1['xmin']<=det2['xmin'] and det1['ymin']<=det2['ymin'] and det1['xmax']>=det2['xmax'] and det1['ymax']>=det2['ymax']):
#    coords = (det1['xmin'], det1['ymin']), det1['xmax'] - det1['xmin'], det1['ymax'] - det1['ymin']
#    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor='r', linewidth=5))
#    print y
#    print str(det1['xmin'])+'\t'+str(det1['ymin'])+'\t'+str(det1['xmax'])+'\t'+str(det1['ymax'])
#elif (det2['xmin']<=det1['xmin'] and det2['ymin']<=det1['ymin'] and det2['xmax']>=det1['xmax'] and det2['ymax']>=det1['ymax']):
#    coords = (det2['xmin'], det2['ymin']), det2['xmax'] - det2['xmin'], det2['ymax'] - det2['ymin']
#    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor='b', linewidth=5))
#    print y2
#    print str(det2['xmin'])+'\t'+str(det2['ymin'])+'\t'+str(det2['xmax'])+'\t'+str(det2['ymax'])
#else:
    coords1 = (det1['xmin'], det1['ymin']), det1['xmax'] - det1['xmin'], det1['ymax'] - det1['ymin']
    currentAxis.add_patch(plt.Rectangle(*coords1, fill=False, edgecolor='r', linewidth=5))
    coords2 = (det2['xmin'], det2['ymin']), det2['xmax'] - det2['xmin'], det2['ymax'] - det2['ymin']
    currentAxis.add_patch(plt.Rectangle(*coords2, fill=False, edgecolor='b', linewidth=5))
    print y
    print str(det1['xmin'])+'\t'+str(det1['ymin'])+'\t'+str(det1['xmax'])+'\t'+str(det1['ymax'])
    print y2
    print str(det2['xmin'])+'\t'+str(det2['ymin'])+'\t'+str(det2['xmax'])+'\t'+str(det2['ymax'])
    
    x_init=''
    for q in range(0,len(image_list[0])):
        #print q
        #print lines[1][q]
        if image_list[p][q]=='.':
            break
        elif (image_list[p][q]=='1' or image_list[p][q]=='2' or image_list[p][q]=='3' or image_list[p][q]=='4' or image_list[p][q]=='5' or image_list[p][q]=='6' or image_list[p][q]=='7' or image_list[p][q]=='8' or image_list[p][q]=='9' or image_list[p][q]=='0'):
            x_init+=image_list[p][q]
    x_init=x_init
    print x_init
    
    var=str(x_init)+'\t'+f_ind1[0][0]+'\t'+str(f1[0])+'\t'+str(det1['xmin'])+'\t'+str(det1['ymin'])+'\t'+str(det1['xmax'])+'\t'+str(det1['ymax'])+'\t'+f_ind2[0][0]+'\t'+str(f2[0])+'\t'+str(det2['xmin'])+'\t'+str(det2['ymin'])+'\t'+str(det2['xmax'])+'\t'+str(det2['ymax'])
    a.writerows([[var]])
    
    !rm -rf _temp
    
b.close()

CPU mode
W0430 02:42:28.959839 24436 _caffe.cpp:135] DEPRECATION WARNING - deprecated use of Python interface
W0430 02:42:28.959861 24436 _caffe.cpp:136] Use this instead (with the named "weights" parameter):
W0430 02:42:28.959863 24436 _caffe.cpp:138] Net('/home/ambika/caffe/models/bvlc_reference_rcnn_ilsvrc13/deploy.prototxt', 1, weights='/home/ambika/caffe/models/bvlc_reference_rcnn_ilsvrc13/bvlc_reference_rcnn_ilsvrc13.caffemodel')
I0430 02:42:28.961028 24436 net.cpp:53] Initializing net from parameters: 
name: "R-CNN-ilsvrc13"
state {
  phase: TEST
  level: 0
}
layer {
  name: "data"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 10
      dim: 3
      dim: 227
      dim: 227
    }
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
layer {
  name: "pool1"
  type: "Pooling"
  

/home/ambika/.local/lib/python2.7/site-packages/ipykernel/__main__.py:37: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/ambika/.local/lib/python2.7/site-packages/ipykernel/__main__.py:42: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
/home/ambika/.local/lib/python2.7/site-packages/ipykernel/__main__.py:59: FutureWarning: order is deprecated, use sort_values(...)
/home/ambika/.local/lib/python2.7/site-packages/ipykernel/__main__.py:69: FutureWarning: order is deprecated, use sort_values(...)


CPU mode
W0430 02:43:29.060906 24542 _caffe.cpp:135] DEPRECATION WARNING - deprecated use of Python interface
W0430 02:43:29.060920 24542 _caffe.cpp:136] Use this instead (with the named "weights" parameter):
W0430 02:43:29.060923 24542 _caffe.cpp:138] Net('/home/ambika/caffe/models/bvlc_reference_rcnn_ilsvrc13/deploy.prototxt', 1, weights='/home/ambika/caffe/models/bvlc_reference_rcnn_ilsvrc13/bvlc_reference_rcnn_ilsvrc13.caffemodel')
I0430 02:43:29.062038 24542 net.cpp:53] Initializing net from parameters: 
name: "R-CNN-ilsvrc13"
state {
  phase: TEST
  level: 0
}
layer {
  name: "data"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 10
      dim: 3
      dim: 227
      dim: 227
    }
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
layer {
  name: "pool1"
  type: "Pooling"
  

In [4]:
path='/home/ambika/INF_project/detection_results.txt'
d = np.loadtxt(path, str)

print d

['144114' 'airplane' '1.57993' '75' '152' '345' '288' 'airplane' '0.93291'
 '20' '151' '500' '285']
